In [ ]:
#newenv1
#!/usr/bin/env conda run -n newenv1 python
import sys
print(sys.executable)    ### Shoul be this C:\ProgramData\Anaconda3_1\envs\newenv1\python.exe

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# !{sys.executable} -m pip install tabula <- do not use
# !{sys.executable} -m pip install tabula-py  <- ok
# !{sys.executable} -m pip install PyPDF2
# !{sys.executable} -m pip install tabula-py

In [ ]:
import numpy as np
import tabula
import pandas as pd
import PyPDF2
import re
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [ ]:
path = os.getcwd()+'\\DATA'

files = []
filespath = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.pdf' in file:
            files.append(file)
            filespath.append(os.path.join(r, file))
            
# since there is only one file I need only the file name
os.path.join(r, files[0])
files[:4]

In [ ]:
# funzione per il regex automatico

def findpattern(df,pattern):
    r = re.compile(pattern)
    for i in np.arange(0,df.shape[0],1):
        for e in np.arange(0,df.shape[1],1):
            if r.match(str(str(df.iloc[i,e]))): 
                return df.iloc[i,e],i,e

In [ ]:
df = tabula.read_pdf('C:/Users/fpe/JUPYTER/datimacchina/Reports/HL/Station 23/HL-23-B001E.pdf', pages=[1,3])

In [ ]:
def new_df_row(pdf_file):
    
    file=files[filespath.index(pdf_file)]
    print(pdf_file)
    
    list_err = []
    
    try:
    
        df = tabula.read_pdf(pdf_file, pages=[1,3])

        bra_ls = ['ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR', file]

        for i in np.arange(0,len(df),1):

            for e in df[i].columns:

                if all (x in e for x in ['Building','Code']): 

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

                    nome,row,col = findpattern(df[i],r'.*-.*-')

                    bra_ls[0] = df[i].iloc[row,col]   # build_code
                    bra_ls[1] = df[i].iloc[row,col+1]  # municip
                    bra_ls[2] = df[i].iloc[row,col+2]  # street
                    bra_ls[3] = df[i].iloc[row,col+3]  # st_num

                if 'Distance to tunnel.1' in e:

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

                    bra_ls[4] = df[i].iloc[-1,df[i].columns.get_loc(e)]         # Distance tunnel.1
                    bra_ls[5] = df[i].iloc[-1,df[i].columns.get_loc(e)-1]       # Distance tunnel
                    bra_ls[6] = df[i].iloc[-1,df[i].columns.get_loc(e)-2]       # Position
                    bra_ls[7] = df[i].iloc[-1,df[i].columns.get_loc(e)-3]       # Chainage

                if 'Distance to far' in e:

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

                    bra_ls[4] = df[i].iloc[-1,df[i].columns.get_loc(e)]         # Distance tunnel.1
                    bra_ls[5] = df[i].iloc[-1,df[i].columns.get_loc(e)-1]       # Distance tunnel
                    bra_ls[6] = df[i].iloc[-1,df[i].columns.get_loc(e)-2]       # Position
                    bra_ls[7] = df[i].iloc[-1,df[i].columns.get_loc(e)-3]       # Chainage
                    
                    
                if 'Distance to Axis 1' in e:    

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))


                    for ii in reversed(range(0,df[i].shape[1])): 

                        if (str(df[i].iloc[-1,ii]).isalpha()==False) or (str(df[i].iloc[-1,ii])=='On'): 

                            bra_ls[4] = df[i].iloc[-1,ii]               # Distance tunnel.1
                            bra_ls[5] = df[i].iloc[-1,ii-1]             # Distance tunnel
                            bra_ls[6] = df[i].iloc[-1,ii-2]             # Position
                            bra_ls[7] = df[i].iloc[-1,ii-3]             # Chainage
                            break                    
                    

                if any (x in e for x in ['of floors below','No of floors below','Building length']):

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

                    for ii in reversed(range(0,df[i].shape[1])): 

                        if (str(df[i].iloc[-1,ii]).isalpha()==False) or (str(df[i].iloc[-1,ii])=='N/A'): 

                            bra_ls[8] = df[i].iloc[-1,ii]               # No. of floors below
                            bra_ls[9] = df[i].iloc[-1,ii-1]             # No. of floors above
                            bra_ls[10] = df[i].iloc[-1,ii-2]             # width
                            bra_ls[11] = df[i].iloc[-1,ii-3]             # Lenght
                            break
                    

                if 'Constructive elements' in e:

                    #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))


                    bra_ls[12] = df[i].iloc[-1,df[i].columns.get_loc(e)]         # Constructive elements
                    bra_ls[13] = df[i].iloc[-1,df[i].columns.get_loc(e)-1]       # Building foundations          


        ## Find the category

        object = PyPDF2.PdfFileReader(pdf_file)        
        NumPages = object.getNumPages()

        for i in range(0, NumPages):
            PageObj = object.getPage(i)
            Text = PageObj.extractText() 


            try:
                posit = re.search(r"Category:", Text).start()

                print ('La posizione alla pagina {} {}'.format(i,posit))

                if 'edium' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Medium'
                elif 'ood' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Good'
                elif 'ritical' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Critical'
                elif 'Med' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Medium'
                elif 'Goo' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Good'
                elif 'Crit' in Text[posit:posit+25]: 
                    bra_ls[14] = 'Critical'
                else:
                    bra_ls[14] = 'Na'
            except:
                pass

        print (bra_ls)
        print('\n\n')
        
        
        if 'ERR' in bra_ls: list_err.append(pdf_file)
        
        return bra_ls,list_err
    
    except:
        
        print('An error has occurred')
        
        bra_ls = ['ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR', file]
        
        
        print (bra_ls)
        print('\n\n')
        
        list_err.append(pdf_file)
        
        return bra_ls,list_err

    
    

In [ ]:
df1 = pd.DataFrame(columns=['build_code','municip','street','st_num','dist_tnl_1','dist_tnl','posit','chain',
                            'floor_b','floor_a','width','lenght'
                            ,'constr_elem','build_info','Category','pdfname'])


for file_path_ in filespath[50:52]:

    row_ls = new_df_row(file_path_)[0]
    
    df1.loc[len(df1)]=row_ls
    
    
df1    

In [ ]:
df1.to_excel('db_all.xls')

### Check errrors

In [ ]:
# df1[df1.floor_b.str.contains(NaN)]

df1[df1['dist_tnl_1'].isnull()]

### Codice per prova

In [ ]:
pdf_file = [s for s in filespath if "LD-45-B003" in s][0]

df = tabula.read_pdf(pdf_file, pages=[1,3])

bra_ls = ['ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR','ERR']

for i in np.arange(0,len(df),1):

    for e in df[i].columns:

        if all (x in e for x in ['Building','Code']): 

            #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

            nome,row,col = findpattern(df[i],r'.*-.*-')

            bra_ls[0] = df[i].iloc[row,col]   # build_code
            bra_ls[1] = df[i].iloc[row,col+1]  # municip
            bra_ls[2] = df[i].iloc[row,col+2]  # street
            bra_ls[3] = df[i].iloc[row,col+3]  # st_num             
        
        
        
        
        if any (x in e for x in ['Distance to tunnel.1','Distance to Axis 1','Distance to far']):

            #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))
            
                           
            for ii in reversed(range(0,df[i].shape[1])): 

                if (str(df[i].iloc[-1,ii]).isalpha()==False) or (str(df[i].iloc[-1,ii])=='On'): 
                    
                    bra_ls[4] = df[i].iloc[-1,ii]               # Distance tunnel.1
                    bra_ls[5] = df[i].iloc[-1,ii-1]             # Distance tunnel
                    bra_ls[6] = df[i].iloc[-1,ii-2]             # Position
                    bra_ls[7] = df[i].iloc[-1,ii-3]             # Chainage
                    break
                   
                    

        if any (x in e for x in ['of floors below','No of floors below','Building length']):

            #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

            for ii in reversed(range(0,df[i].shape[1])): 

                if (str(df[i].iloc[-1,ii]).isalpha()==False) or (str(df[i].iloc[-1,ii])=='N/A'): 
                    
                    bra_ls[8] = df[i].iloc[-1,ii]               # No. of floors below
                    bra_ls[9] = df[i].iloc[-1,ii-1]             # No. of floors above
                    bra_ls[10] = df[i].iloc[-1,ii-2]             # width
                    bra_ls[11] = df[i].iloc[-1,ii-3]             # Lenght
                    break


        if 'Constructive element' in e:

            #print("Found: '{}'".format(e),' at dataframe: df[{}] and col_loc:{}'.format(i,df[i].columns.get_loc(e)))

            for ii in reversed(range(0,df[i].shape[1])): 

                if 'aterial' in str(df[i].iloc[-2,ii]): 
                    
                    bra_ls[12] = df[i].iloc[-1,ii]              # Constructive elements
                    bra_ls[13] = df[i].iloc[-1,ii-1]             # Building foundations
                    break                     


## Find the category




object = PyPDF2.PdfFileReader(pdf_file)        
NumPages = object.getNumPages()

for i in range(0, NumPages):
    PageObj = object.getPage(i)
    Text = PageObj.extractText() 


    try:
        posit = re.search(r"Category:", Text).start()

        print ('La posizione alla pagina {} {}'.format(i,posit))

        if 'edium' in Text[posit:posit+25]: 
            bra_ls[14] = 'Medium'
        elif 'ood' in Text[posit:posit+25]: 
            bra_ls[14] = 'Good'
        elif 'ritical' in Text[posit:posit+25]: 
            bra_ls[14] = 'Critical'
        elif 'Med' in Text[posit:posit+25]: 
            bra_ls[14] = 'Medium'
        elif 'Goo' in Text[posit:posit+25]: 
            bra_ls[14] = 'Good'
        elif 'Crit' in Text[posit:posit+25]: 
            bra_ls[14] = 'ritical'
        elif 'ood' in Text[posit:posit+25]: 
            bra_ls[14] = 'Good'
        elif 'CRIT' in Text[posit:posit+25]: 
            bra_ls[14] = 'Critical'
        elif 'MED' in Text[posit:posit+25]: 
            bra_ls[14] = 'Medium'   
        elif 'OOD' in Text[posit:posit+25]: 
            bra_ls[14] = 'Good'        
        else:
            bra_ls[14] = 'ERR2'
    except:
        pass

print (bra_ls)



In [ ]:
['build_code','municip','street','st_num','dist_tnl_1','dist_tnl','posit','chain','floor_b','floor_a','width','lenght','constr_elem','build_info','Category']

In [ ]:
Text

In [ ]:
bra_ls

In [ ]:
Text[posit:posit+25]

In [ ]:
df[3]

In [ ]:
df[4].shape[1]

In [ ]:
for ii in reversed(range(0,df[4].shape[1])):   
    if str(df[4].iloc[-1,ii]).isalpha()==False:
        print(df[4].iloc[-1,ii])
        print(ii)
        break

In [ ]:
str('5').isnumeric()

In [ ]:
str(df[4].iloc[-1,6]).isalpha()

In [ ]:
str(df[4].iloc[-1,4])

In [ ]:
float('n/a')

In [ ]:
df[4]

In [ ]:
pdf_file = "C:\\Users\\fpe\\JUPYTER\\datimacchina\\Reports\\HL\\Station 21\\HL-21-B001.pdf"


object = PyPDF2.PdfFileReader(pdf_file)        
NumPages = object.getNumPages()

for i in range(0, NumPages):
    
    PageObj = object.getPage(i)
    Text = PageObj.extractText() 

    try:
        posit = re.search(r"Category:", Text).start()

        print ('La posizione alla pagina {} {}'.format(i,posit))

        if 'edium' in Text[posit:posit+25]: bra_ls.append('Medium')
        if 'ood' in Text[posit:posit+25]: bra_ls.append('Good')
        if 'ritical' in Text[posit:posit+25]: bra_ls.append('Critical')
        if 'Med' in Text[posit:posit+25]: bra_ls.append('Medium')
        if 'Goo' in Text[posit:posit+25]: bra_ls.append('Good')
        if 'Crit' in Text[posit:posit+25]: bra_ls.append('Critical')
            
    except:
        
        pass


In [ ]:
bra_ls[50] = 'Azz'
bra_ls